[딥 러닝을 이용한 자연어 처리 입문(바로가기)](https://wikidocs.net/30707)을 정리한 내용입니다.

더 알아봐야 할 것
- svd_model.components_ : SVD에서 VT행렬
- .get_feature_names() : 카운트 기반 단어표현에서 vocabulary 리턴

# 토픽 모델링
- **문서 집합의 추상적인 주제를 발견**하기 위한 모델링
- 텍스트 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 텍스트 마이닝 기법

# LSA개요
- TF-IDF와 같은 빈도수 기반 표현방법은 단어의 의미를 고려하지 못한다.
- 따라서, 잠재된(Latent)의미를 이끌어내는 방법으로 LSA가 사용되었다.(SVD에 대한 이해 필요)
- 즉, DTM이나 TF-IDF행렬에 truncated SVD를 사용해서 차원을 축소, 잠재적 의미를 이끌어낸다는 아이디어


## 1) 일반적 SVD(Full SVD)
- m * n 행렬 A를 다음과 같이 분해하는 것.
$$A = USV^T$$

$$U: m x m 직교행렬(AA^T = U(SS^T)U^T)$$
$$V: n x n 직교행렬(A^TA = V(S^TS)V^T)$$
$$S: m x n 직사각  대각행렬$$
- 행렬Σ의 주대각원소를 행렬 A의 특이값이라고 하며, 내림차순으로 정렬되어있다.

## 2) Truncated SVD
- LSA의 경우 SVD에서 나온 행렬들을 일부 삭제하여 사용한다.
     - 설명력이 낮은 정보 삭제 -> 계산비용 낮춤
- Σ의 대각원소의 값 중 상위 t개의 값만 남긴다.
    - t는 찾고자 하는 토픽의 수
    - 크게잡으면 다양한 의미 가져갈 수 있지만 노이즈 존재
- 절단시키면 값의 손실이 일어나 A행렬로 복원할 수 없다.


Σ에서 t개의 topics만 선택했다고 해보자.  
> U: m x t -> 문서 개수 x 토픽의 수  
VT: t x n -> 토픽의 수 x 단어 개수의 크기  
Σ : t x t  

각 행렬들이 위와 같은 사이즈로 결정될 것이다.

- U의 각 행은 잠재 의미를 표현하기 위한 수치화된 각각의 문서 벡터  

- VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터

## LSA 실습
sklearn의 20newsgroups 데이터셋 사용하기



In [1]:
import pandas as pd
import pickle
from sklearn.datasets import fetch_20newsgroups
with open('c:/data/news.data', 'rb') as f:
    dataset  = pickle.load(f)

documents = dataset.data
len(documents)

11314

In [2]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [3]:
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### 텍스트 전처리
- 알파벳 제외한 구두점, 숫자, 특수문제 제거
- 길이 짧은 단어 제거
- 모든 알파벳 소문자로 바꾸기

In [4]:
news_df = pd.DataFrame({'document':documents})

news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# str로 바꿔주지 않으면 \n과 같은 이스케이프 코드를 삭제하지 않는다.

news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# lambda 함수부분: 한 행씩 x로 받아서 공백을 기준으로 나눈 리스트를 만들고, 그 리스트의 요소(단어 하나씩)마다 길이가 3이면 문자열로 합친뒤, 그 값으로 초기화한다.
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [5]:
news_df

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased what disagre...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...",well will have change scoring playoff pool unf...
...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...",danny rubenstein israeli journalist will speak...
11310,\n,
11311,\nI agree. Home runs off Clemens are always m...,agree home runs clemens always memorable kinda...
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet with orange micros grappler syste...


In [6]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [7]:
# 불용어 제거
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].map(lambda x: x.split())
tokenized_doc = tokenized_doc.map(lambda x: [item for item in x if item not in stop_words])
# tokenized_doc = map(lambda x : [x를 이용한 리스트 만들기]) -> x로 리스트를 만들어 리턴함

In [8]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [12]:
# TfidfVectorizer는 토큰화되어있지 않은 텍스트를 입력으로 사용한다.
# 따라서 TF-IDF 행렬 만들기 위해 역토큰화(Detokenization)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
news_df['clean_doc'] = detokenized_doc

In [14]:
news_df

,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure story seem biased disagree statement...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...
2,Although I realize that principle is not one o...,although realize principle strongest points wo...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss proposal much ...
4,"Well, I will have to change the scoring on my ...",well change scoring playoff pool unfortunately...
...,...,...
11309,"Danny Rubenstein, an Israeli journalist, will ...",danny rubenstein israeli journalist speaking t...
11310,\n,
11311,\nI agree. Home runs off Clemens are always m...,agree home runs clemens always memorable kinda...
11312,I used HP DeskJet with Orange Micros Grappler ...,used deskjet orange micros grappler system upd...


In [15]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
                             max_features = 1000, 
                             max_df = 0.5,
                            # vocabulary 만들때 document frequency가 설정한 값보다 높으면 무시한다.(디폴트 1.0)
                             smooth_idf = True)
                             # document frequencies에 1을 더한다. -> 0으로 나누는 것을 방지(디폴트 True)
    
X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape

(11314, 1000)

In [16]:
tokenized_doc

0        [well, sure, story, seem, biased, disagree, st...
1        [yeah, expect, people, read, actually, accept,...
2        [although, realize, principle, strongest, poin...
3        [notwithstanding, legitimate, fuss, proposal, ...
4        [well, change, scoring, playoff, pool, unfortu...
                               ...                        
11309    [danny, rubenstein, israeli, journalist, speak...
11310                                                   []
11311    [agree, home, runs, clemens, always, memorable...
11312    [used, deskjet, orange, micros, grappler, syst...
11313    [argument, murphy, scared, hell, came, last, y...
Name: clean_doc, Length: 11314, dtype: object

In [22]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized',n_iter=100,random_state=122)
svd_model.fit(X)
len(svd_model.components_)
# svd_model.components_는 VT에 해당된다.
# 정확하게 토픽의 수 t x 단어의 수의 크기를 가진다.

20

In [32]:
# Tfidf = TfidfVectorizer(max_features=1000).get_feature_names

# get_feature_names()가 모든 vectorizer에서 사용되는지 알아보기위한 시실험

In [37]:
svd_model.components_.shape

(20, 1000)

In [41]:
terms = vectorizer.get_feature_names()
# .get_feature_names()로 단어집합 term 객체 생성.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic 2: [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic 3: [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic 4: [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic 5: [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic 6: [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic 7: [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic 8: [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic 9: [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]
Topic 10

# LDA
> 문서 집합으로부터 어떤 토픽이 존재하는지 알아내기 위한 알고리즘

LDA의 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생선한다고 가정한다.

데이터가 주어지면 LDA는 문서가 생성되던 과정을 역추적한다.

- LDA를 수행할 때, 문서 집합에서 토픽이 몇 개가 존재할지 가정하는것은 분석자의 몫.
- **각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정**  
(문서-토픽 & 토픽-단어)

- DTM or TF-IDF 행렬을 입력으로 한다. 즉, **단어의 순서는 신경쓰지 않겠다**는 의미.

## LDA 수행

1. 분석자가 토픽 개수(k) 지정

2. 모든 단어를 k개 중 하나의 토픽에 할당
    - 하나의 토픽을 랜덤으로 할당. -> 각 문서는 토픽을 가진다.(랜덤 할당이기에 틀린 상태)

3. (반복) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있다고 가정
w를 아래 두 기준에 따라서 토픽을 재할당.
    - p(topic t | document d): 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
    - p(word w | topic t): 단어 w를 갖고 있는 모든 문서들 중 토픽 t가 할당된 비율
    
예) 두 문서가 있을 때, 문서1의 한 단어(A)의 토픽을 결정한다고 하자.
- 문서1의 단어들이 어떤 토픽에 해당하는지 확인, A가 속할 토픽들의 분포 확인
- A가 전체 문서에서 어떤 토픽에 할당되어져 있는지 확인.


## LSA vs LDA
LSA: DTM을 차원 축소하여 근접 단어들을 토픽으로 묶는다.  
LDA: 단어가 특정 토픽에 존재할 확률과, 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출

# 실습

In [48]:
# LSA실습에서 만든 tokenized_doc을 활용.
tokenized_doc

0        [well, sure, story, seem, biased, disagree, st...
1        [yeah, expect, people, read, actually, accept,...
2        [although, realize, principle, strongest, poin...
3        [notwithstanding, legitimate, fuss, proposal, ...
4        [well, change, scoring, playoff, pool, unfortu...
                               ...                        
11309    [danny, rubenstein, israeli, journalist, speak...
11310                                                   []
11311    [agree, home, runs, clemens, always, memorable...
11312    [used, deskjet, orange, micros, grappler, syst...
11313    [argument, murphy, scared, hell, came, last, y...
Name: clean_doc, Length: 11314, dtype: object

각 단어에 정수 인코딩 + 각 뉴스에서의 단어의 빈도수를 기록최종적으로 doc2bow를 이용하여 (word_id, word_frequency)의 형태로 바꾸고자 한다.

먼저 corpora.Dictionary()를 사용하여 (인덱스: 해당 단어)의 딕셔너리를 구해준다.

In [55]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
#tornnized_doc(Series객체)의 각 단어들을 dictionary(인덱스:단어)를 활용하여 doc2bow(단어 인덱스:해당 단어 빈도)로 만들어준다.
print(corpus[1]) # 두 번째 뉴스 결과 출력

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [54]:
# (52,1)은 인덱스번호 52를 가지는 단어가 두번째 뉴스에서 1번 등장했다는 의미이다.
print('위에서 52의 인덱스를 가지는 단어는' ,dictionary[52],'이다.')

위에서 52의 인덱스를 가지는 단어는 well 이다.


### LDA 모델 훈련시키기

In [59]:
import gensim
NUM_TO_PICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = NUM_TO_PICS,
                                          id2word = dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
# passes : 알고리즘의 동작 횟수 - 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 된다.
# num_words : 출력하고자 하는 단어의 수.

In [60]:
for topic in topics:
    print(topic)

(0, '0.013*"would" + 0.011*"time" + 0.010*"like" + 0.009*"think"')
(1, '0.014*"file" + 0.010*"available" + 0.010*"files" + 0.010*"window"')
(2, '0.016*"pitt" + 0.016*"gordon" + 0.015*"pitching" + 0.015*"banks"')
(3, '0.016*"picture" + 0.011*"sleeve" + 0.010*"rockefeller" + 0.008*"switzerland"')
(4, '0.016*"would" + 0.016*"people" + 0.007*"think" + 0.006*"government"')
(5, '0.010*"jesus" + 0.007*"believe" + 0.007*"would" + 0.007*"people"')
(6, '0.008*"information" + 0.007*"public" + 0.005*"encryption" + 0.005*"national"')
(7, '0.009*"like" + 0.008*"bike" + 0.007*"right" + 0.007*"much"')
(8, '0.011*"eisa" + 0.007*"corp" + 0.007*"ieee" + 0.006*"astros"')
(9, '0.016*"armenian" + 0.013*"turkish" + 0.012*"armenians" + 0.008*"russian"')
(10, '0.079*"drive" + 0.050*"scsi" + 0.042*"disk" + 0.030*"hard"')
(11, '0.007*"good" + 0.006*"also" + 0.005*"price" + 0.005*"mail"')
(12, '0.027*"game" + 0.024*"team" + 0.018*"games" + 0.017*"play"')
(13, '0.024*"char" + 0.015*"filename" + 0.008*"borland" + 0

## 문서 별 토픽 분포 보기

In [63]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i == 5:
        break
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(4, 0.4731768), (9, 0.14595816), (16, 0.36714587)]
1 번째 문서의 topic 비율은 [(0, 0.34306315), (4, 0.17499757), (5, 0.28510228), (6, 0.12306397), (9, 0.027841998), (18, 0.02747191)]
2 번째 문서의 topic 비율은 [(0, 0.107579134), (1, 0.060708635), (4, 0.66533196), (16, 0.15280618)]
3 번째 문서의 topic 비율은 [(0, 0.21825007), (1, 0.024551325), (4, 0.34481), (6, 0.21188332), (10, 0.03379682), (11, 0.115184665), (17, 0.041183032)]
4 번째 문서의 topic 비율은 [(0, 0.36672553), (3, 0.22471875), (12, 0.3392332), (14, 0.03968223)]


### 보기 편하게 df 형태로 만들어보기
(나중에 추가하기)

# LDA sklearn 실습

In [64]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [66]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


In [70]:
text = data[['headline_text']]
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [71]:
import nltk
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

C:\Users\moon\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [72]:
print(text.head(5))

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


In [73]:
# 불용어 제거
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in stop])

C:\Users\moon\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
print(text.head())

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


In [77]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos = 'v') for word in x])
print(text.head(5))

                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


C:\Users\moon\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
# 길이가 3이하인 단어에 제거
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word)>3])
print(tokenized_doc[:5])

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


## TF-IDF 행렬 만들기

In [82]:
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
text['headline_text'] = detokenized_doc

C:\Users\moon\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [83]:
text['headline_text'][:5]

0       decide community broadcast licence
1       fire witness must aware defamation
2    call infrastructure protection summit
3                   staff aust strike rise
4      strike affect australian travellers
Name: headline_text, dtype: object

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 1000)
X = vectorizer.fit_transform(text['headline_text'])
X.shape

(1082168, 1000)

## 토픽 모델링

In [92]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=10,
                                      learning_method='online',
                                      random_state=777, max_iter=1)
# learning_method : components_를 학습하기 위한 방법을 설정하는 옵션.
# 데이터 사이즈가 많을 경우 online 방법이 빠르다.
# batch : 각각의 EM 업데이트마다 모든 훈련데이터 사용
# online : 각각의 EM 업데이트마다 mini-batch 방식 사용.
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

[[1.00001783e-01 1.00001935e-01 1.00009183e-01 ... 1.00003242e-01
  1.00002451e-01 1.00005588e-01]
 [1.00001784e-01 1.12967298e+03 1.00010368e-01 ... 1.00004030e-01
  1.00004859e-01 7.42418314e+02]
 [1.00002549e-01 1.00001501e-01 3.48345083e+03 ... 1.00003412e-01
  1.00001950e-01 1.00004703e-01]
 ...
 [1.00000863e-01 1.00001435e-01 1.00002551e-01 ... 1.00003941e-01
  1.00001945e-01 1.00002981e-01]
 [1.00002460e-01 1.00002580e-01 1.00010173e-01 ... 1.00003458e-01
  1.00001385e-01 1.00006822e-01]
 [1.00000306e-01 1.00001164e-01 1.00004623e-01 ... 1.00003728e-01
  1.00001927e-01 1.00003424e-01]]
(10, 1000)


In [95]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print('Topic %d:' % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n -1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('queensland', 7581.82), ('change', 5732.86), ('2016', 5468.84), ('coast', 5351.31), ('year', 5350.75)]
Topic 2: [('government', 8547.73), ('court', 7404.45), ('south', 6558.01), ('world', 6553.8), ('home', 5492.63)]
Topic 3: [('kill', 5754.15), ('report', 5479.61), ('rural', 5470.91), ('school', 5340.98), ('jail', 4539.3)]
Topic 4: [('attack', 6863.65), ('first', 5464.93), ('state', 4835.11), ('turnbull', 4171.82), ('time', 3739.29)]
Topic 5: [('call', 7866.35), ('death', 5854.35), ('miss', 4381.62), ('women', 4166.18), ('accuse', 4007.25)]
Topic 6: [('trump', 11711.23), ('charge', 8288.93), ('murder', 6205.86), ('take', 5746.65), ('warn', 5014.09)]
Topic 7: [('australia', 13293.85), ('find', 8700.7), ('canberra', 5966.55), ('show', 5812.48), ('open', 5540.06)]
Topic 8: [('police', 11841.04), ('australian', 10854.93), ('sydney', 8265.66), ('interview', 5949.19), ('brisbane', 4832.47)]
Topic 9: [('melbourne', 7416.33), ('house', 5905.61), ('live', 5352.98), ('could', 5169.57)